이 코드는 영수증 이미지에서 코너를 찾아 perspective transformation을 적용하는 완전한 파이프라인을 구현합니다. 주요 기능은 다음과 같습니다:

1. 영수증 감지:

    - 이미지 전처리 (그레이스케일, 블러, 엣지 검출)
    - 윤곽선 검출
    - 4개의 코너 포인트 추출


2. Perspective Transform:

    - 감지된 코너를 기반으로 투시 변환 수행
    - 목표 크기로 변환
    - 선택적으로 랜덤한 변형 적용 가능


3. 시각화:

    - 감지된 코너와 윤곽선 표시
    - 변환 전후 이미지 저장

In [ ]:
import cv2
import numpy as np
from typing import Tuple, List, Optional
import random

class ReceiptPerspectiveTransform:
    """영수증 이미지의 코너 감지 및 perspective transformation을 수행하는 클래스"""
    
    def __init__(self, 
                 target_size: Tuple[int, int] = (800, 1000),
                 random_state: Optional[int] = None):
        """
        Args:
            target_size: 변환된 영수증의 목표 크기 (width, height)
            random_state: 랜덤 시드값
        """
        self.target_size = target_size
        if random_state is not None:
            random.seed(random_state)
            np.random.seed(random_state)
    
    def _preprocess_image(self, image: np.ndarray) -> np.ndarray:
        """이미지 전처리"""
        # 그레이스케일 변환
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        # 가우시안 블러
        blurred = cv2.GaussianBlur(gray, (5, 5), 0)
        # 엣지 검출
        edges = cv2.Canny(blurred, 75, 200)
        # 모폴로지 연산으로 엣지 보강
        kernel = np.ones((5,5), np.uint8)
        dilated = cv2.dilate(edges, kernel, iterations=1)
        return dilated
    
    def _find_receipt_contour(self, preprocessed: np.ndarray) -> np.ndarray:
        """영수증의 윤곽선 검출"""
        # 윤곽선 찾기
        contours, _ = cv2.findContours(
            preprocessed, 
            cv2.RETR_EXTERNAL, 
            cv2.CHAIN_APPROX_SIMPLE
        )
        
        # 가장 큰 윤곽선 선택
        if not contours:
            raise ValueError("No contours found in the image")
            
        largest_contour = max(contours, key=cv2.contourArea)
        return largest_contour
    
    def _get_corner_points(self, contour: np.ndarray) -> np.ndarray:
        """윤곽선에서 4개의 코너 포인트 추출"""
        # 근사 다각형 구하기
        epsilon = 0.02 * cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, epsilon, True)
        
        # 4개의 코너가 아닌 경우 처리
        if len(approx) != 4:
            # 윤곽선의 경계 상자 사용
            rect = cv2.minAreaRect(contour)
            box = cv2.boxPoints(rect)
            approx = np.int0(box)
        
        # 코너 포인트 정렬 (좌상단부터 시계방향)
        pts = approx.reshape(4, 2)
        rect = np.zeros((4, 2), dtype=np.float32)
        
        s = pts.sum(axis=1)
        rect[0] = pts[np.argmin(s)]  # 좌상단
        rect[2] = pts[np.argmax(s)]  # 우하단
        
        diff = np.diff(pts, axis=1)
        rect[1] = pts[np.argmin(diff)]  # 우상단
        rect[3] = pts[np.argmax(diff)]  # 좌하단
        
        return rect
    
    def _apply_perspective_transform(self, 
                                   image: np.ndarray, 
                                   corners: np.ndarray,
                                   random_transform: bool = False) -> Tuple[np.ndarray, np.ndarray]:
        """투시 변환 적용"""
        width, height = self.target_size
        
        # 목표 코너 포인트
        dst_points = np.array([
            [0, 0],
            [width-1, 0],
            [width-1, height-1],
            [0, height-1]
        ], dtype=np.float32)
        
        if random_transform:
            # 랜덤한 변형 추가
            margin = min(width, height) * 0.1
            noise = np.random.uniform(-margin, margin, dst_points.shape)
            dst_points += noise
        
        # 변환 행렬 계산
        matrix = cv2.getPerspectiveTransform(corners, dst_points)
        
        # 변환 적용
        transformed = cv2.warpPerspective(image, matrix, (width, height))
        
        return transformed, matrix
    
    def transform(self, 
                 image: np.ndarray,
                 random_transform: bool = False) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
        """
        영수증 감지 및 투시 변환 수행
        
        Args:
            image: 입력 이미지
            random_transform: 랜덤한 변형 적용 여부
            
        Returns:
            transformed_image: 변환된 이미지
            corners: 감지된 코너 좌표
            transform_matrix: 변환 행렬
        """
        # 전처리
        preprocessed = self._preprocess_image(image)
        
        # 영수증 윤곽선 검출
        contour = self._find_receipt_contour(preprocessed)
        
        # 코너 포인트 추출
        corners = self._get_corner_points(contour)
        
        # 투시 변환 적용
        transformed, matrix = self._apply_perspective_transform(
            image, corners, random_transform
        )
        
        return transformed, corners, matrix
    
    def visualize_detection(self, 
                          image: np.ndarray,
                          corners: np.ndarray) -> np.ndarray:
        """감지 결과 시각화"""
        vis = image.copy()
        
        # 코너 포인트 그리기
        for point in corners:
            x, y = point.astype(int)
            cv2.circle(vis, (x, y), 5, (0, 255, 0), -1)
        
        # 윤곽선 그리기
        pts = corners.reshape((-1, 1, 2)).astype(np.int32)
        cv2.polylines(vis, [pts], True, (0, 255, 0), 2)
        
        return vis

def demo_transform():
    """변환 데모"""
    # 이미지 로드
    image = cv2.imread('receipt.jpg')
    
    # 변환기 초기화
    transformer = ReceiptPerspectiveTransform(target_size=(800, 1000))
    
    # 변환 수행
    transformed, corners, matrix = transformer.transform(image)
    
    # 결과 시각화
    detection_vis = transformer.visualize_detection(image, corners)
    
    # 결과 저장
    cv2.imwrite('receipt_detection.jpg', detection_vis)
    cv2.imwrite('receipt_transformed.jpg', transformed)

if __name__ == '__main__':
    demo_transform()

In [ ]:
# 변환기 초기화
transformer = ReceiptPerspectiveTransform(target_size=(800, 1000))

# 기본 변환
transformed, corners, matrix = transformer.transform(image)

# 랜덤 변형 적용
transformed_random, corners, matrix = transformer.transform(
    image, 
    random_transform=True
)

# 감지 결과 시각화
detection_vis = transformer.visualize_detection(image, corners)

개선할 수 있는 부분들:

코너 감지 정확도 향상을 위한 추가적인 전처리 방법 적용
다양한 형태의 랜덤 변형 추가 (회전, 스케일링 등)
영수증이 아닌 경우의 예외 처리 강화

필요한 기능이나 수정하고 싶은 부분이 있으시다면 말씀해 주세요.